<a href="https://colab.research.google.com/github/jlab-sensing/MFC_Modeling/blob/main/SNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  SNN Models
###  In order to run the code in this notebook, you must download the `ucscMFCDataset` directory and `stanfordMFCDataset.zip`, which expands into the directory `rocket4`, from [Hugging Face](https://huggingface.co/datasets/adunlop621/Soil_MFC/tree/main), and store them in the same directory as this notebook. You can also find several pretrained models in the at this link, with the naming conventions described in the [README](https://github.com/jlab-sensing/MFC_Modeling#:~:text=Repository%20files%20navigation-,README,-MFC_Modeling)

In [30]:
%pip install --upgrade hepml
%pip install arrow
%pip install keras_lr_finder
%pip install pandas
%pip install snntorch --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [31]:
# reload modules before executing user code
#%load_ext autoreload
# reload all modules every time before executing Python code
#%autoreload 2
# render plots in notebook

# Misc imports
%matplotlib inline
import datetime
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from hepml.core import plot_regression_tree
sns.set(color_codes=True)
sns.set_palette(sns.color_palette("muted"))
import random
import statistics

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

# torch imports
import torch
import torch.nn as nn

# snnTorch imports
import snntorch as snn
from snntorch import functional as SF
import snntorch.spikeplot as splt

# # keras imports
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras import backend as K



##  Load and Format Dataset 1

### Remember to download `stanfordMFCDataset.zip`, which expands into the directory `rocket4`, from [Hugging Face](https://huggingface.co/datasets/adunlop621/Soil_MFC/tree/main), and store it in the same directory as this notebook before executing the following code.

In [120]:
#Load teros data
import glob
teros_files = glob.glob("rocket4/TEROSoutput*.csv")
X = pd.DataFrame()
for f in teros_files:
  try:
    csv = pd.read_csv(f, index_col=False).dropna()
    X = pd.concat([X, csv])
  except:
    continue

In [121]:
#Load power data
power_files = glob.glob("rocket4/soil*.csv")
y = pd.DataFrame()
for f in sorted(power_files, key=lambda x: int(x.split('.')[0].split('_')[-1])):
#in power_files:
  try:
    csv = pd.read_csv(f, on_bad_lines='skip', skiprows=10).dropna(how='all')
    csv = csv.rename({'Unnamed: 0': 'timestamp'}, axis='columns')
    y = pd.concat([y,csv])
  except:
    continue
y["timestamp"] = y["timestamp"].round(decimals = 1)

In [122]:
#Convert current to amps, voltage to volts
y["I1L [10pA]"] = np.abs(y["I1L [10pA]"] * 1E-11)
y["V1 [10nV]"] = np.abs(y["V1 [10nV]"] * 1E-8)
y["I1H [nA]"] = np.abs(y["I1H [nA]"] * 1E-9)

In [123]:
#Sort data by timestamp, convert to datetime
X = X.sort_values(['timestamp'])
y = y.sort_values(['timestamp'])
X['timestamp'] = pd.to_datetime(X['timestamp'], unit='s')
y['timestamp'] = pd.to_datetime(y['timestamp'], unit='s')

#Merge data by timestamp
uncut_df = pd.merge_asof(left=X,right=y,direction='nearest',tolerance=pd.Timedelta('1 sec'), on = 'timestamp').dropna(how='all')

#Isolate data from cell0
df = uncut_df.loc[uncut_df['sensorID'] == 0]

#Localize timestamp
df.timestamp = df.timestamp.dt.tz_localize('UTC').dt.tz_convert('US/Pacific')

/var/folders/1g/jt7x_x9n6j1f3317_dwv3k9m0000gn/T/ipykernel_36727/3455547848.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.timestamp = df.timestamp.dt.tz_localize('UTC').dt.tz_convert('US/Pacific')


In [124]:
#Use only data from after deployment date
#df = df.loc[(df['timestamp'] > '2021-09-24') & (df['timestamp'] < '2021-10-15')] #Future of Clean Computing Graph
#df = df.loc[(df['timestamp'] > '2021-06-24') & (df['timestamp'] < '2021-07-02')]
#df = df.loc[(df['timestamp'] > '2021-06-18')] #Two weeks after deployment
df = df.loc[(df['timestamp'] > '2021-06-04')] #Deployment date
#df = df.loc[(df['timestamp'] > '2021-06-25') & (df['timestamp'] < '2021-06-26')] #Small training set

#Power drop
#df = df.loc[(df['timestamp'] > '2021-11-01') & (df['timestamp'] < '2021-11-22')]

#Drop data outages
df = df.drop(df[(df.timestamp > '2021-11-11') & (df.timestamp < '2021-11-22 01:00:00')].index)
df = df.drop(df[(df.timestamp > '2022-01-27')].index)
#df = df.set_index('timestamp')
df = df[:-1]

In [125]:
df = df.set_index('timestamp')

In [126]:
#Get time since deployement
df['tsd'] = (df.index - df.index[0]).days
df['hour'] = (df.index).hour

In [127]:
#Calculate power
df["power"] = np.abs(np.multiply(df.iloc[:, 7], df.iloc[:, 8]))
#df["power"] = np.abs(np.multiply(df["I1L [10pA]"], df["V1 [10nV]"]))

#Convert to nW
df['power'] = df['power']*1E9

In [128]:
#Convert to 10 nanoamps, 10 microvolts
df["I1L [10pA]"] = np.abs(df["I1L [10pA]"] * 1E8)
df["V1 [10nV]"] = np.abs(df["V1 [10nV]"] * 1E5)
df["I1H [nA]"] = np.abs(df["I1H [nA]"] * 1E8)

In [129]:
df = df.reset_index()

In [130]:
#Add power time series
df['power - 1h'] = df['power'].shift(1).dropna()
df['power - 2h'] = df['power'].shift(2).dropna()
df['power - 3h'] = df['power'].shift(3).dropna()
#df['power - 2h'] = df['power'].shift(2).dropna()
#df['previous_power - 3'] = df['power'].shift(3).dropna()
#df['previous_power - 4'] = df['power'].shift(4).dropna()

#Add teros time series
df['EC - 1h'] = df['EC'].shift(1).dropna()
df['EC - 2h'] = df['EC'].shift(2).dropna()
df['EC - 3h'] = df['EC'].shift(3).dropna()

df['temp - 1h'] = df['temp'].shift(1).dropna()
df['temp - 2h'] = df['temp'].shift(2).dropna()
df['temp - 3h'] = df['temp'].shift(3).dropna()

df['raw_VWC - 1h'] = df['raw_VWC'].shift(1).dropna()
df['raw_VWC - 2h'] = df['raw_VWC'].shift(2).dropna()
df['raw_VWC - 3h'] = df['raw_VWC'].shift(3).dropna()

#Add voltage and current time series
df['V1 - 1h'] = df['V1 [10nV]'].shift(1).dropna()
df['V1 - 2h'] = df['V1 [10nV]'].shift(2).dropna()
df['V1 - 3h'] = df['V1 [10nV]'].shift(3).dropna()

df['I1L - 1h'] = df['I1L [10pA]'].shift(1).dropna()
df['I1L - 2h'] = df['I1L [10pA]'].shift(2).dropna()
df['I1L - 3h'] = df['I1L [10pA]'].shift(3).dropna()

df['I1H - 1h'] = df['I1H [nA]'].shift(1).dropna()
df['I1H - 2h'] = df['I1H [nA]'].shift(2).dropna()
df['I1H - 3h'] = df['I1H [nA]'].shift(3).dropna()
df = df.dropna()

In [131]:
#df = df.rename(columns={'power': 'power [μW]'})
df = df.rename(columns={'I1L [10pA]': 'Current (uA)', 'V1 [10nV]' : 'Voltage (mV)', 'power' : 'Power (uW)'})
df = df.set_index('timestamp')

In [132]:
#New runtime calculation
import math
from dateutil import parser
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

def internal_R_v3(R=2000): #return internal resistance of v3 cells in ohms
    #https://www.jstage.jst.go.jp/article/jwet/20/1/20_21-087/_pdf
    v0_oc = 48.5e-3 #48.5 mV
    v0_cc = 4.8e-3
    v0_r = R*((v0_oc/v0_cc)-1)

    v1_oc = 43.8e-3
    v1_cc = 20.9e-3
    v1_r = R*((v1_oc/v1_cc)-1)

    v2_oc = 45.2e-3
    v2_cc = 23.5e-3
    v2_r = R*((v2_oc/v2_cc)-1)

    return (v0_r+v1_r+v2_r)/3

def internal_R_v0(R=2000): #return internal resistance of v0 cells in ohms
    v3_oc = 41.7e-3 #41.7mV
    v3_cc = 5.1e-3
    v3_r = R*((v3_oc/v3_cc)-1)

    v4_oc = 48.7e-3
    v4_cc = 16.8e-3
    v4_r = R*((v4_oc/v4_cc)-1)

    v5_oc = 39.1e-3
    v5_cc = 16.9e-3
    v5_r = R*((v5_oc/v5_cc)-1)

    return (v3_r+v4_r+v5_r)/3

def SMFC_current(v, R):
    return v/R

#MODEL
def cap_leakage(E_cap_tn, timestep):
    #Spec for KEMET T491
    return 0.01e-6 * E_cap_tn * timestep

def Matrix_Power(V, R):
    #efficiency interpolated from https://www.analog.com/media/en/technical-documentation/data-sheets/ADP5091-5092.pdf
    #given I_in = 100 uA and SYS = 3V
    #V is the voltage (V) of the SMFC we captured
    #R is the resistance (ohms) of the load we used to get that voltage trace
    #Eta = -292.25665*V**4 + 784.30311*V**3 - 770.71691*V**2 + 342.00502*V + 15.83307
    #Eta = Eta/100
    Eta = 0.60
    Pmax = (V**2)/R
    Pout = Eta*Pmax
    #assert((Eta > 0) & (Eta < 1))
    #assert(Pout < 12000e-6)
    return Pout

def update_capEnergy(e0, V_applied, R, C, dt):
    # e0: initial energy stored
    # V_applied: voltage from SMFC
    # R: internal resistance of SMFC
    # C: capacitance of capacitor
    # dt: time step since last data point
    e_cap = e0 + Matrix_Power(V_applied, R)*dt - cap_leakage(e0, dt)
    v_cap = math.sqrt(2*e_cap/C)
    if e_cap < 0: #Not charging if leakage is greater than energy
        e_cap = 0

    return e_cap, v_cap #output final e and v

def Advanced_energy():
    #Now representing "Advanced"
    #startup time of 2500 ms
    t = 2500e-3
    e = 2.4 * 128e-3 * t
    e_startup = 2.4 * 128e-3 * 5e-3
    return e+e_startup

def Minimal_energy():
    #Now representing "Minimal"
    t = 0.888e-3 #tentative time
    e = 0.9 * 4.8e-3 * t #this uses average current
    e_startup = 0#assume negligible, no known startup time given
    return  e + e_startup

def Analog_energy():
    #Now representing Analog
    t = 1e-3 #estimated operating time
    e = 0.11 * 2.15e-6 * t
    e_startup = 0 #analog device, no startup needed :)
    return e + e_startup

#STEP 3:
# For each day:
#   on_Minimal, on_Advanced, on_Analog = 0
#   For each time step (like every 60 s given our logging freq):
#       - Update the energy in our capacitor (put fcn in models.py) given (1) input voltage, (2) time step, (3) capacitance (prob 10 uF), this will be an integral
#       - Check if energy is enough to turn on (1) 1 uJ load, (2) 10 uJ load, and (3) 20 uJ load (will tweak later to reflect real energy cost of each system)
#       - If so, add to on_Minimal, on_Advanced, and on_Analog and reset capacitor energy to 0 J (might tweak this value)
#   Append on_Minimal, on_Advanced, on_Analog to on_Minimal_list, on_Advanced_list, on_Analog_list. This will be a list of how many sensor readings we are able to take with each of these systems every day given the energy we got
#STEP 4: Visualize the daily # of readings with 3 bar graphs, y axis is # of readings and x axis is days.
#   - Given 3 lists of integer values, plot them on bar graphs

def group_util(test_date1, test_date2, N):
    diff = (test_date2 - test_date1) / N
    return [test_date1 + diff * idx for idx in range(N)] + [test_date2]

def oracle_simulate(v_list, C_h):
    #Calculate maximum energy
    total_E = 0
    for i in range(len(v_list) - 1):
        t = (v_list.index[i+1] - v_list.index[i]).total_seconds()
        if t > 180:
          print("Discontinuity")
          print(v_list.index[i+1], v_list.index[i])
          print(v_list['Voltage (mV)'][i+1], v_list['Voltage (mV)'][i])
          #total_E, ignore = update_capEnergy(total_E, V_applied=(v_list['V1 [mV]'][i+1] + v_list['V1 [mV]'][i])/2, R=internal_R_v0(), C=C_h[0], dt = t)
        else:
          total_E, ignore = update_capEnergy(total_E, V_applied=max(v_list['Voltage (mV)'][i], v_list['Voltage (mV)'][i+1]), R=internal_R_v0(), C=C_h[0], dt = t)
    print("Oracle activations:", math.floor(total_E/Minimal_energy()))
    return(math.floor(total_E/Minimal_energy()))

def naive_simulate(t_list, v_list, v_list_naive, v_list_fine, C_h):
    # t_list: list of decimal time stamps in unit of days (e.g. 71.85893518518519 day), same length as v_list
    # v_list: list of voltage values from SFMC
    # C_h: capacitance of the capacitor being filled up by harvester

    #assume capacitor is completely discharged at start
    e_minimal_stored = 0
    e_minimal_stored_theo = 0

    #Initialize evaluation metrics
    false_act = 0
    max_act = 0
    pred_act = 0
    succ_act = 0

    total_E = 0
    total_E_naive = 0

    #Calculate maximum energy
    #for i in range(len(v_list_fine) - 1):
    #    t = (v_list_fine.index[i+1] - v_list_fine.index[i]).total_seconds()
    #    total_E, ignore = update_capEnergy(total_E, V_applied=v_list_fine['V1 [10nV]'][i], R=internal_R_v0(), C=C_h[0], dt = t)
    #print(total_E/Minimal_energy())
    v = v_list_naive.mean()
    #for each voltage data point
    for jj in range(len(v_list) - 1): #last data point was at 71.85893518518519 day
        t = (v_list.index[jj+1] - v_list.index[jj]).total_seconds()
        if t <= time_frame_seconds:
          #Total predicted vs. actual energy stored
          #Predict energy stored during scheduled sub-interval
          total_E, ignore = update_capEnergy(total_E, V_applied=v_list[jj], R=internal_R_v0(), C=C_h[0], dt = t)
          total_E_naive, ignore = update_capEnergy(total_E_naive, V_applied=v, R=internal_R_v0(), C=C_h[0], dt = t)

          E_Minimal_pred, v_minimal_pred = update_capEnergy(e_minimal_stored, V_applied=v, R=internal_R_v0(), C=C_h[0], dt = t) #set dt as length of prediction interval, in seconds
          pred_act += math.floor(E_Minimal_pred/Minimal_energy()) #Update number of activations predicted
          itn = 0
          if math.floor(E_Minimal_pred/Minimal_energy()) > 0:
              minimal_intervals = [date for date in group_util(v_list.index[jj], v_list.index[jj] + timedelta(seconds=t), math.floor(E_Minimal_pred/Minimal_energy()))]
              #Calculate desired interval
              int_len = time_frame_seconds /  math.floor(E_Minimal_pred/Minimal_energy())
              for i in range(len(minimal_intervals) - 1):
                  #Determine actual energy stored during scheduled sub-interval
                  start = v_list_fine.index.searchsorted(minimal_intervals[i])
                  end =  v_list_fine.index.searchsorted(minimal_intervals[i+1])

                  E_Minimal, ignore = update_capEnergy(e_minimal_stored, V_applied=v_list_fine.iloc[start:end]['Voltage (mV)'].mean(), R=internal_R_v0(), C=C_h[0], dt = int_len)
                  if not math.isnan(v_list_fine.iloc[start:end]['Voltage (mV)'].mean()):
                    if E_Minimal < Minimal_energy():
                        false_act += 1
                        e_minimal_stored = max(0, E_Minimal - Minimal_energy())
                        itn += 1

                    elif E_Minimal >= Minimal_energy():
                        succ_act += 1
                        e_minimal_stored = max(0, E_Minimal - Minimal_energy())
                        itn+= 1

                    else:
                      print('Error')
                      print(e_minimal_stored, v)

                  #Unit test
                  #else:
                  #  print("?")
                  #  print(v_list_fine.index[start])
                  #  print(v_list_fine.index[end])
                  #  print(minimal_intervals[i], minimal_intervals[i+1])

              #Unit test
              #if itn != math.floor(E_Minimal_pred/Minimal_energy()):
              #    print("itn not matching")
              #    print(itn, math.floor(E_Minimal_pred/Minimal_energy()))
              #    continue

          else:
              e_minimal_stored, ignore = update_capEnergy(e_minimal_stored, V_applied=v_list[jj], R=internal_R_v0(), C=C_h[0], dt = t)
              #Added this
              #start = v_list_fine.index.searchsorted(v_list.index[jj])
              #end =  v_list_fine.index.searchsorted(v_list.index[jj+1])
              #for h in range(start, end):
              #    v = v_list_fine.iloc[h]['V1 [mV]']
              #    interval_length = ((v_list_fine.index[h+1]) - (v_list_fine.index[h])).total_seconds()
              #    E_Minimal, ignore = update_capEnergy(e_minimal_stored, V_applied=v, R=internal_R_v0(), C=C_h[0], dt = interval_length)
              #    e_minimal_stored = E_Minimal


        else:
          print("It's over 9000!", v_list.index[jj], v_list.index[jj+1])

    print("Naive total_E activations:", total_E/Minimal_energy())
    print("Naive total_E_pred activations:", total_E_naive/Minimal_energy())
    return pred_act, false_act, succ_act, total_E_naive

def getMax(c_list, input_list):
    max_value = max(input_list)
    i = [index for index, item in enumerate(input_list) if item == max_value][0]
    return i, max_value, c_list[i]


#SMFC
import csv
from collections import defaultdict
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
from datetime import datetime

def butter_lowpass(cutoff, fs, order=5):
        return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def getMFC_data(y_test, test_pred):
    unix_time = y_test.index
    d0 = unix_time[0]
    days = []
    for d in unix_time:
        day = d
        day_from_start = day-d0
        decimal_day = day_from_start.total_seconds()/(24 * 3600)
        days.append(decimal_day)

    return days

def simulate(t_list, v_list, v_list_pred, v_list_fine, C_h):
    # t_list: list of decimal time stamps in unit of days (e.g. 71.85893518518519 day), same length as v_list
    # v_list: list of voltage values from SFMC
    # C_h: capacitance of the capacitor being filled up by harvester

    #assume capacitor is completely discharged at start
    e_minimal_stored = 0
    e_minimal_stored_theo = 0

    #Initialize evaluation metrics
    false_act = 0
    max_act = 0
    pred_act = 0
    succ_act = 0

    total_E = 0
    total_E_pred = 0

    #Calculate maximum energy
    #for i in range(len(v_list_fine) - 1):
    #    t = (v_list_fine.index[i+1] - v_list_fine.index[i]).total_seconds()
    #    total_E, ignore = update_capEnergy(total_E, V_applied=v_list_fine['V1 [10nV]'][i], R=internal_R_v0(), C=C_h[0], dt = t)
    #print(total_E/Minimal_energy())
    #for each voltage data point
    for jj in range(len(v_list) - 1): #last data point was at 71.85893518518519 day
        t = (v_list.index[jj+1] - v_list.index[jj]).total_seconds()
        total_E, ignore = update_capEnergy(total_E, V_applied=v_list[jj], R=internal_R_v0(), C=C_h[0], dt = t)
        total_E_pred, ignore = update_capEnergy(total_E_pred, V_applied=v_list_pred[jj], R=internal_R_v0(), C=C_h[0], dt = t)
        if t <= time_frame_seconds:
          #Total predicted vs. actual energy stored
          #Predict energy stored during scheduled sub-interval
          E_Minimal_pred, v_minimal_pred = update_capEnergy(e_minimal_stored, V_applied=v_list_pred[jj], R=internal_R_v0(), C=C_h[0], dt = t) #set dt as length of prediction interval, in seconds
          pred_act += math.floor(E_Minimal_pred/Minimal_energy()) #Update number of activations predicted
          itn = 0
          if math.floor(E_Minimal_pred/Minimal_energy()) > 0:
              minimal_intervals = [date for date in group_util(v_list_pred.index[jj], v_list_pred.index[jj] + timedelta(seconds=t), math.floor(E_Minimal_pred/Minimal_energy()))]
              #Calculate desired interval
              int_len = time_frame_seconds /  math.floor(E_Minimal_pred/Minimal_energy())
              for i in range(len(minimal_intervals) - 1):
                  #Determine actual energy stored during scheduled sub-interval
                  start = v_list_fine.index.searchsorted(minimal_intervals[i])
                  end =  v_list_fine.index.searchsorted(minimal_intervals[i+1])
                  v = v_list_fine.iloc[start:end]['Voltage (mV)'].mean()

                  #interval_length = ((v_list_fine.index[end]) - (v_list_fine.index[start])).total_seconds()
                  #if interval_length > int_len:
                  #  print('interval_length > int_len')
                  #  print('interval_length, int_len:', interval_length, int_len)
                  #  print(v_list_fine.index[start], v_list_fine.index[end])
                  #else:
                  #  print('interval_length <= int_len')
                  #  print('interval_length, int_len:', interval_length, int_len)
                  #  print(v_list_fine.index[start], v_list_fine.index[end])

                  E_Minimal, ignore = update_capEnergy(e_minimal_stored, V_applied=v, R=internal_R_v0(), C=C_h[0], dt = int_len)
                  if not math.isnan(v_list_fine.iloc[start:end]['Voltage (mV)'].mean()):
                    if E_Minimal < Minimal_energy():
                        false_act += 1
                        e_minimal_stored = max(0, E_Minimal - Minimal_energy())
                        itn += 1

                    elif E_Minimal >= Minimal_energy():
                        succ_act += 1
                        e_minimal_stored = max(0, E_Minimal - Minimal_energy())
                        itn+= 1

                    else:
                      print('Error')
                      print(e_minimal_stored, v)

                  #Unit test
                  #else:
                  #  print("?")
                  #  print(v_list_fine.index[start])
                  #  print(v_list_fine.index[end])
                  #  print(minimal_intervals[i], minimal_intervals[i+1])

              #Unit test
              #if itn != math.floor(E_Minimal_pred/Minimal_energy()):
              #    print("itn not matching")
              #    print(itn, math.floor(E_Minimal_pred/Minimal_energy()))
              #    continue

          else:
              e_minimal_stored, ignore = update_capEnergy(e_minimal_stored, V_applied=v_list[jj], R=internal_R_v0(), C=C_h[0], dt = t)
              #Added this
              #start = v_list_fine.index.searchsorted(v_list.index[jj])
              #end =  v_list_fine.index.searchsorted(v_list.index[jj+1])
              #for h in range(start, end):
              #    v = v_list_fine.iloc[h]['V1 [mV]']
              #    interval_length = ((v_list_fine.index[h+1]) - (v_list_fine.index[h])).total_seconds()
              #    E_Minimal, ignore = update_capEnergy(e_minimal_stored, V_applied=v, R=internal_R_v0(), C=C_h[0], dt = interval_length)
              #    e_minimal_stored = E_Minimal


        else:
          print("It's over 9000!", v_list.index[jj], v_list.index[jj+1])

    print("Runtime total_E activations:", total_E/Minimal_energy())
    print("Runtime total_E_pred activations:", total_E_pred/Minimal_energy())
    return pred_act, false_act, succ_act, total_E, total_E_pred

## Specify Device so we can use GPU

In [133]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Network Architecture

In [134]:
beta = 0.9

# old design network
# model = Sequential()
# model.add(LSTM(200, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(3))
# model.compile(loss=quantile_loss, metrics=['mape'], optimizer='adam')

# Define Network
class Net(nn.Module):
    def __init__(self, num_inputs, num_steps):
        super().__init__()

        self.num_inputs = num_inputs
        self.num_steps = num_steps

        num_hidden1 = 200

        # layer 1
        self.slstm1 = snn.SLSTM(num_inputs, num_hidden1, threshold = 0.25)

        # layer 2
        self.fc1 = torch.nn.Linear(in_features=num_hidden1, out_features=100)
        self.lif1 = snn.Leaky(beta=beta, threshold = 0.5)

        # randomly initialize decay rate for output neuron
        beta_out = random.uniform(0.5, 1)

        # layer 2
        self.fc2 = torch.nn.Linear(in_features=100, out_features=3)
        self.lif2 = snn.Leaky(beta=beta_out, learn_beta=True, reset_mechanism="none")


    def forward(self, x):
        # Initialize hidden states and outputs at t=0
        syn1, mem1 = self.slstm1.reset_mem()
        mem2 = self.lif1.reset_mem()
        mem3 = self.lif2.reset_mem()

        # Record the final layer
        spk1_rec = []
        spk2_rec = []
        spk3_rec = []
        mem_rec = []

        for step in range(self.num_steps):
            spk1, syn1, mem1 = self.slstm1(x.flatten(1), syn1, mem1)
            spk2, mem2 = self.lif1(self.fc1(spk1), mem2)
            spk3, mem3 = self.lif2(self.fc2(spk2), mem3)

            # Append the Spike and Membrane History
            spk1_rec.append(spk1)
            spk2_rec.append(spk2)
            spk3_rec.append(spk3)
            mem_rec.append(mem3)

        return torch.stack(spk1_rec), torch.stack(spk2_rec), torch.stack(spk3_rec), torch.stack(mem_rec)

In [57]:
from sklearn.model_selection import TimeSeriesSplit
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras import backend as K
from torch.utils.data import DataLoader, TensorDataset

In [58]:
X_train, X_test = train_test_split(pd.concat([df["power - 1h"], df["power - 2h"], df["power - 3h"], df["V1 - 1h"], df["V1 - 2h"], df["V1 - 3h"], df["I1L - 1h"], df["I1L - 2h"], df["I1L - 3h"],df["EC - 1h"], df["EC - 2h"], df["EC - 3h"], df["raw_VWC - 1h"], df["raw_VWC - 2h"], df["raw_VWC - 3h"], df["temp - 1h"], df["temp - 2h"], df["temp - 3h"], df["tsd"], df["hour"]], axis = 1), test_size=0.3, shuffle=False)
y_train, y_test = train_test_split(pd.concat([df['Power (uW)'], df['Voltage (mV)'], df['Current (uA)']], axis = 1), test_size=0.3, shuffle=False)

In [59]:
X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)

In [60]:
#All Data (For Type 1 and 2 Models)
X = pd.concat([df["power - 1h"], df["power - 2h"], df["power - 3h"], df["V1 - 1h"], df["V1 - 2h"], df["V1 - 3h"], df["I1L - 1h"], df["I1L - 2h"], df["I1L - 3h"],df["EC - 1h"], df["EC - 2h"], df["EC - 3h"], df["raw_VWC - 1h"], df["raw_VWC - 2h"], df["raw_VWC - 3h"], df["temp - 1h"], df["temp - 2h"], df["temp - 3h"], df["tsd"], df["hour"]], axis = 1)

#Electricity Data Omitted (For Type 1A and 2A Models)
#X = pd.concat([df["EC - 1h"], df["EC - 2h"], df["EC - 3h"], df["raw_VWC - 1h"], df["raw_VWC - 2h"], df["raw_VWC - 3h"], df["temp - 1h"], df["temp - 2h"], df["temp - 3h"], df["tsd"], df["hour"]], axis = 1)

#Environmental Data Omitted (For Type 1B and 2B Models)
#X = pd.concat([df["power - 1h"], df["power - 2h"], df["power - 3h"], df["V1 - 1h"], df["V1 - 2h"], df["V1 - 3h"], df["I1L - 1h"], df["I1L - 2h"], df["I1L - 3h"]], axis = 1)

y = pd.concat([df["Power (uW)"], df['Voltage (mV)'], df['Current (uA)']], axis = 1)

print("Rows " + str(X.shape[0]) + " Columns " + str(X.shape[1]))

print("Rows " + str(y.shape[0]) + " Columns " + str(y.shape[1]))

Rows 969652 Columns 20
Rows 969652 Columns 3


## 2. Train and Load Models

###  2.1 Train new SNN model

In [ ]:
power_mape = []
voltage_mape = []
current_mape = []

E_actual_list = []
E_pred_list = []

max_act_list = []
pred_act_list = []
succ_act_list = []

pred_act_naive_list = []
false_act_naive_list = []
succ_act_naive_list = []

#Set parameters
batchsize_list = [300, 150, 50, 20, 8]
time_frame_list = ['3min', '5min', '15min', '30min', '60min']
time_frame_seconds_list = [180, 300, 900, 1800, 3600]
n = 0

for j in range(len(batchsize_list)):
    n += 1
    if n == 2: #Select which timescales to train for
        #Normalize Data
        X_normalized = ((X - X.min()) / (X.max() - X.min()))

        #Split train and test sets
        X_train, X_test = train_test_split(X_normalized, test_size=0.3, shuffle=False)
        y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False)

        X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
        y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)

        batchsize = batchsize_list[j]
        time_frame = time_frame_list[j]
        time_frame_seconds = time_frame_seconds_list[j]

        print(time_frame)

        #Resample data
        X_train = X_train.resample(time_frame).mean().dropna()
        X_valid = X_valid.resample(time_frame).mean().dropna()
        X_test = X_test.resample(time_frame).mean().dropna()

        y_train = y_train.resample(time_frame).mean().dropna()
        y_valid = y_valid.resample(time_frame).mean().dropna()
        y_test = y_test.resample(time_frame).mean().dropna()

        #Reshape data
        X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_valid = X_valid.values.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
        X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

        # convert to tensor
        X_train = torch.tensor(X_train)
        y_train = torch.tensor(y_train.values)
        X_test = torch.tensor(X_test)
        y_test = torch.tensor(y_test.values)

        # make datasets
        train_dataset = TensorDataset(X_train, y_train)
        test_dataset = TensorDataset(X_test, y_test)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

        # Define the number of time steps for the spiking
        num_steps = 50
        num_inputs = X_train.shape[2]

        # create new inctance of the SNN Class
        model = Net(num_inputs, num_steps).to(device)

        # define optimizer
        optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

        # define loss function
        def quantile_loss(y_true, y_pred, quantile=0.5):
            error = y_true - y_pred
            loss = torch.mean(torch.max(quantile * error, (quantile - 1) * error))
            return loss
        loss_fn = quantile_loss
        #loss_fn = torch.nn.MSELoss()
        #loss_fn = torch.nn.L1Loss()

        # initialize histories
        loss_hist = []
        avg_loss_hist = []
        acc_hist = []
        mape_hist = []
        num_epochs = 25

        # put model into train mode
        model.train()

        # Train Loop
        for epoch in range(num_epochs):
            for i, (data, targets) in enumerate(iter(train_loader)):
                # move to device
                data = data.to(device)
                targets = targets.to(device)

                # change to floats
                data = data.float()
                targets = targets.float()

                # run forward pass
                _, _, _, mem = model(data)
                # calculate loss
                loss_val = loss_fn(mem[-1], targets)

                # calculate and store MAPE Loss
                mem_numpy = mem.cpu().detach().numpy()
                #mem_numpy = mem.detach().numpy()
                targets_numpy = targets.cpu().detach().numpy()
                #targets_numpy = targets.detach().numpy()
                mape_hist.append(MAPE(mem_numpy[-1], targets_numpy))
                power_mape.append(MAPE(mem_numpy[-1][:,0], targets_numpy[:,0]))
                voltage_mape.append(MAPE(mem_numpy[-1][:,1], targets_numpy[:,1]))
                current_mape.append(MAPE(mem_numpy[-1][:,2], targets_numpy[:,2]))

                # Gradient calculation + weight update
                optimizer.zero_grad()
                loss_val.backward()
                optimizer.step()

                # Store loss history for future plotting
                loss_hist.append(loss_val.item())

                if i%10 == 0:
                    print(f"Epoch {epoch}, Iteration {i} Train Loss: {loss_val.item():.2f}")
                if len(loss_hist) > 100:
                    avg_loss_hist.append(sum(loss_hist[-100:])/len(loss_hist[-100:]))
                else:
                    avg_loss_hist.append(0)

            if len(loss_hist) > 100:
                print(f'New Epoch! Avg loss for the last 100 iterations: {avg_loss_hist[-1]}')

3min
Epoch 0, Iteration 0 Train Loss: 10971.40
Epoch 0, Iteration 10 Train Loss: 915.95
Epoch 0, Iteration 20 Train Loss: 762.81
Epoch 0, Iteration 30 Train Loss: 1777.55
Epoch 0, Iteration 40 Train Loss: 1382.86
Epoch 0, Iteration 50 Train Loss: 902.26
Epoch 0, Iteration 60 Train Loss: 1050.10
Epoch 0, Iteration 70 Train Loss: 1094.10
Epoch 0, Iteration 80 Train Loss: 1182.41
Epoch 0, Iteration 90 Train Loss: 931.53
Epoch 0, Iteration 100 Train Loss: 834.55
Epoch 0, Iteration 110 Train Loss: 732.78
Epoch 0, Iteration 120 Train Loss: 791.05
Epoch 0, Iteration 130 Train Loss: 764.48
Epoch 0, Iteration 140 Train Loss: 524.42
Epoch 0, Iteration 150 Train Loss: 443.54
Epoch 0, Iteration 160 Train Loss: 457.31
Epoch 0, Iteration 170 Train Loss: 485.84
New Epoch! Avg loss for the last 100 iterations: 742.8388580322265
Epoch 1, Iteration 0 Train Loss: 10677.97
Epoch 1, Iteration 10 Train Loss: 606.98
Epoch 1, Iteration 20 Train Loss: 441.39
Epoch 1, Iteration 30 Train Loss: 1435.31
Epoch 1, I

In [ ]:
#Save model
checkpoint = {'state_dict': model.state_dict(),'optimizer' :optimizer.state_dict()}
torch.save(checkpoint, 'snn_30min_quant50.pth')
%mv snn_30min_quant50.pth 'trained_models'

###  2.2 TSRV on Type 1 Models

In [135]:
print(df.columns)

Index(['sensorID', 'raw_VWC', 'temp', 'EC', 'I1L_valid', 'I2L_valid',
       'I1H [nA]', 'Current (uA)', 'Voltage (mV)', 'V2 [10nV]', 'I2H [nA]',
       'I2L [10pA]', 'tsd', 'hour', 'Power (uW)', 'power - 1h', 'power - 2h',
       'power - 3h', 'EC - 1h', 'EC - 2h', 'EC - 3h', 'temp - 1h', 'temp - 2h',
       'temp - 3h', 'raw_VWC - 1h', 'raw_VWC - 2h', 'raw_VWC - 3h', 'V1 - 1h',
       'V1 - 2h', 'V1 - 3h', 'I1L - 1h', 'I1L - 2h', 'I1L - 3h', 'I1H - 1h',
       'I1H - 2h', 'I1H - 3h'],
      dtype='object')


In [64]:
power_mape = []
voltage_mape = []
current_mape = []

E_actual_list = []
E_pred_list = []

max_act_list = []
pred_act_list = []
succ_act_list = []

pred_act_naive_list = []
false_act_naive_list = []
succ_act_naive_list = []

#Set parameters
batchsize = 8
time_frame = '60min'
time_frame_seconds = 3600
n = 0
splits = TimeSeriesSplit(n_splits=4)
for train_index, test_index in splits.split(X):
    n += 1
    if n >= 1:
        #Split train and test sets
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]

        X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
        y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)

        #Set dataset bounds
        train_bound_lower = y_train.index[0]
        train_bound_upper = y_train.index[-1]
        valid_bound_lower = y_valid.index[0]
        valid_bound_upper = y_valid.index[-1]
        test_bound_lower = y_test.index[0]
        test_bound_upper = y_test.index[-1]

        #Resample data
        X_train = X_train.resample(time_frame).mean().dropna()
        X_valid = X_valid.resample(time_frame).mean().dropna()
        X_test = X_test.resample(time_frame).mean().dropna()

        y_train = y_train.resample(time_frame).mean().dropna()
        y_valid = y_valid.resample(time_frame).mean().dropna()
        y_test = y_test.resample(time_frame).mean().dropna()

        #Reshape data
        X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_valid = X_valid.values.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
        X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

        # convert to tensor
        X_train = torch.tensor(X_train)
        y_train_index_labels = y_train.index
        y_train_column_labels = y_train.columns
        y_train = torch.tensor(y_train.values)

        X_test = torch.tensor(X_test)
        y_test_index_labels = y_test.index
        y_test_column_labels = y_test.columns
        y_test = torch.tensor(y_test.values)

        # make datasets
        train_dataset = TensorDataset(X_train, y_train)
        test_dataset = TensorDataset(X_test, y_test)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

        # Define the number of time steps for the spiking
        num_steps = 50
        num_inputs = X_train.shape[2]

        # create new inctance of the SNN Class
        model = Net(num_inputs, num_steps).to(device)

        # define optimizer
        optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

        # define loss function
        def quantile_loss(y_true, y_pred, quantile=0.05):
            error = y_true - y_pred
            loss = torch.mean(torch.max(quantile * error, (quantile - 1) * error))
            return loss
        loss_fn = quantile_loss

        # initialize histories
        loss_hist = []
        avg_loss_hist = []
        acc_hist = []
        num_epochs = 25

        # put model into train mode
        model.train()

        # Train Loop
        for epoch in range(num_epochs):
            for i, (data, targets) in enumerate(iter(train_loader)):
                # move to device
                data = data.to(device)
                targets = targets.to(device)

                # change to floats
                data = data.float()
                targets = targets.float()

                _, _, _, mem = model(data)

                loss_val = loss_fn(mem[-1], targets)

                # Gradient calculation + weight update
                optimizer.zero_grad()
                loss_val.backward()
                optimizer.step()

                # Store loss history for future plotting
                loss_hist.append(loss_val.item())

                # Update loss plot
                # update_loss_plot(loss_hist)

                if i%10 == 0:
                    print(f"Epoch {epoch}, Iteration {i} Train Loss: {loss_val.item():.2f}")
                if len(loss_hist) > 100:
                    avg_loss_hist.append(sum(loss_hist[-100:])/len(loss_hist[-100:]))
                else:
                    avg_loss_hist.append(0)

            if len(loss_hist) > 100:
                print(f'New Epoch! Avg loss for the last 100 iterations: {avg_loss_hist[-1]}')

        # predictions
        model.eval()

        train_pred = []
        test_pred = []
        with torch.no_grad():
            for data, target in train_dataset:
                data = data.to(device)
                data = data.float()
                _, _, _, train_mem = model(data)
                train_pred.append(train_mem[-1])

            for data, target in test_dataset:
                data = data.to(device)
                data = data.float()
                _, _, _, test_mem = model(data)
                test_pred.append(test_mem[-1])

        # Convert to numpy
        train_pred = torch.cat(train_pred, dim=0)
        train_pred = train_pred.numpy()
        test_pred = torch.cat(test_pred, dim=0)
        test_pred = test_pred.numpy()

        # convert tensors back to dataframe
        y_test = pd.DataFrame(y_test, index=y_test_index_labels, columns=y_test_column_labels)
        y_train = pd.DataFrame(y_test, index=y_train_index_labels, columns=y_test_column_labels)

        #Prepare data for runtime simulation
        y_test['power pred'] = test_pred[:, 0]
        y_test['V1 [mV] pred'] = test_pred[:, 1]
        y_test['I1L [μA] pred'] = test_pred[:, 2]

        days  = getMFC_data(y_test, test_pred)

        v_test = df.loc[(((df.index >= test_bound_lower)) & (df.index <= test_bound_upper))]['Voltage (mV)']
        v_test = v_test.drop(v_test[(v_test.index > '2021-11-11') & (v_test.index < '2021-11-22 01:00:00')].index)
        v_test = pd.DataFrame(v_test)/1E5
        v_avg_true = v_test['Voltage (mV)'].resample(time_frame).mean().dropna()
        v_avg_pred = y_test['V1 [mV] pred']/1E5
        C0 = [0.007000000000000006, 0.007000000000000006, 0.007000000000000006]

        #Remove first and last entries of averaged data to prevent overestimation of available energy
        v_avg_true = v_avg_true[1:][:-1]
        v_avg_pred = v_avg_pred[1:][:-1]

        #Run oracle model
        max_act = oracle_simulate(v_test, C0)

        #Call simulate function
        pred_act, false_act, succ_act, total_E, total_E_pred = simulate(days, v_avg_true, v_avg_pred, v_test, C0)

        #Run naive model
        v_valid = df.loc[(((df.index >= valid_bound_lower)) & (df.index < valid_bound_upper))]['Voltage (mV)']/1E5
        pred_act_naive, false_act_naive, succ_act_naive, total_E = naive_simulate(days, v_avg_true, v_valid, v_test, C0)
        print("Dataset, train set, and test set size:", len(y_train) + len(y_valid) + len(y_test), len(y_train), len(y_test))
        print('Timeframe:', time_frame)

        print('Minimal Application')
        print("Naive vs. DL succesful activations:", succ_act/succ_act_naive)
        #print('Predicted vs. Actual percent difference: %.3f%%' % ((total_E * 100 / total_E_pred) - 100))
        print('Maximum possible activations:', max_act)
        print('Predicted activations:', pred_act)
        print('Successful activations: %d, %.3f%%' % (succ_act, succ_act * 100/pred_act))
        print('Failed activations: %d, %.3f%%' % (false_act, false_act * 100/pred_act))
        print('Missed activations: %d, %.3f%%' % (max_act - succ_act, (max_act - succ_act) * 100/max_act))

        #Naive model
        print('Naive predicted activations (usual actual energy average):', pred_act_naive)
        print('Naive successful activations (usual actual energy average): %d, %.3f%%' % (succ_act_naive, succ_act_naive * 100/pred_act_naive))
        print('Naive failed activations (usual actual energy average): %d, %.3f%%' % (false_act_naive, false_act_naive * 100/pred_act_naive))
        print('Naive missed activations (usual actual energy average): %d, %.3f%%' % (max_act - succ_act_naive, (max_act - succ_act_naive) * 100/max_act))


        print('Voltage overestimation rate: %.3f%%' % ((y_test['Voltage (mV)'].values <= y_test['Voltage (mV)']).mean() * 100))
        print("Test MAPE power: %3f" %  MAPE(y_test['Power (uW)'].values.ravel(), y_test['power pred']))
        print("Test MAPE voltage: %3f" % MAPE(y_test['Voltage (mV)'], y_test['V1 [mV] pred']))
        print("Test MAPE current: %3f" % MAPE(y_test['Current (uA)'], y_test['I1L [μA] pred']))

Epoch 0, Iteration 0 Train Loss: 22023.83
Epoch 0, Iteration 10 Train Loss: 2211.05
Epoch 0, Iteration 20 Train Loss: 1743.63
Epoch 0, Iteration 30 Train Loss: 1186.39
Epoch 0, Iteration 40 Train Loss: 1397.68
Epoch 0, Iteration 50 Train Loss: 1696.61
Epoch 0, Iteration 60 Train Loss: 1203.95
Epoch 0, Iteration 70 Train Loss: 2920.93
Epoch 0, Iteration 80 Train Loss: 1765.85
Epoch 0, Iteration 90 Train Loss: 2137.57
Epoch 1, Iteration 0 Train Loss: 21849.47
Epoch 1, Iteration 10 Train Loss: 1908.69
Epoch 1, Iteration 20 Train Loss: 1313.47
Epoch 1, Iteration 30 Train Loss: 790.27
Epoch 1, Iteration 40 Train Loss: 948.51
Epoch 1, Iteration 50 Train Loss: 1170.54
Epoch 1, Iteration 60 Train Loss: 748.62
Epoch 1, Iteration 70 Train Loss: 2351.21
Epoch 1, Iteration 80 Train Loss: 1193.84
Epoch 1, Iteration 90 Train Loss: 1607.48
New Epoch! Avg loss for the last 100 iterations: 2057.9776861572263
Epoch 2, Iteration 0 Train Loss: 21298.33
Epoch 2, Iteration 10 Train Loss: 1501.35
Epoch 2, It

In [ ]:
# Print out predictions for power, voltage, and current
print("5-Minute-Ahead Predictions:")
print("Power Predictions (uW):")
print(y_test[:, 0])

print("\nVoltage Predictions (mV):")
print(y_test[:, 1])

print("\nCurrent Predictions (μA):")
print(y_test[:, 2])


5-Minute-Ahead Predictions:
Power Predictions (uW):
tensor([405.1541, 341.6363, 381.8097, 378.0867, 448.1771, 309.7523, 319.9965,
        283.5214, 377.2062, 293.7655, 324.5557, 314.3519, 329.7565, 280.7470,
        290.2442, 314.8520, 253.1244, 216.8007, 238.8424, 293.0437, 278.7045,
        247.5437, 247.1386, 288.8400, 263.1261, 318.4699, 305.4739, 341.9922,
        306.8654, 314.7618, 264.4440, 270.7533, 300.5112, 277.7155, 306.7275,
        278.6054, 260.1407, 259.2924, 272.8766, 306.1858, 299.8885, 256.6909,
        233.9902, 271.3538, 282.8519, 285.3684, 323.8132, 355.6323, 314.4305,
        308.6498, 336.6819, 356.4979, 323.8845, 364.1196, 340.2905, 317.3364,
        339.4746, 308.2276, 345.0291, 294.6420, 294.0039, 309.0331, 289.7521,
        284.8598, 247.1115, 284.5787, 253.3137, 270.9416, 258.9177, 285.2174,
        327.3894, 334.6517, 345.3905, 343.8805, 347.8802, 350.3924, 316.4610,
        347.2957, 342.1163, 330.5285, 331.4545, 330.0189, 288.2295, 275.7054,
        276.

##  3. Graph Selected Data

###  3.1 Load and graph pretrained SNN models

###  In order to use pretrained models it is neccesary to download the ```trained_models``` directory from [Hugging Face](https://huggingface.co/datasets/adunlop621/Soil_MFC/tree/main) and store it in the same directory as this notebook

In [ ]:
from keras.models import load_model

time_frame = '60min'
batchsize = 8

X = pd.concat([df["power - 1h"], df["power - 2h"], df["power - 3h"], df["V1 - 1h"], df["V1 - 2h"], df["V1 - 3h"], df["I1L - 1h"], df["I1L - 2h"], df["I1L - 3h"],df["EC - 1h"], df["EC - 2h"], df["EC - 3h"], df["raw_VWC - 1h"], df["raw_VWC - 2h"], df["raw_VWC - 3h"], df["temp - 1h"], df["temp - 2h"], df["temp - 3h"], df["tsd"], df["hour"]], axis = 1)
#X = pd.concat([df["power - 1h"], df["power - 2h"], df["power - 3h"], df["V1 - 1h"], df["V1 - 2h"], df["V1 - 3h"], df["I1L - 1h"], df["I1L - 2h"], df["I1L - 3h"], df["tsd"], df["hour"]], axis = 1)
y = pd.concat([df["Power (uW)"], df['Voltage (mV)'], df['Current (uA)']], axis = 1)

#Normalize Data
X_normalized = ((X - X.min()) / (X.max() - X.min()))

#Split train and test sets
X_train, X_test = train_test_split(X_normalized, test_size=0.3, shuffle=False)
y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False)

X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)

#Resample data

X_valid = X_valid.resample(time_frame).mean().dropna()
y_valid = y_valid.resample(time_frame).mean().dropna()

X_test = X_test.resample(time_frame).mean().dropna()
y_test = y_test.resample(time_frame).mean().dropna()

#Define mv1
mv1 = y_valid

#Reshape data
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_valid = X_valid.values.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert to tensor
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train.values)
X_valid = torch.tensor(X_valid)
y_valid = torch.tensor(y_valid.values)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test.values)

# make datasets
train_dataset = TensorDataset(X_train, y_train)
valid_dataset = TensorDataset(X_valid, y_valid)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batchsize, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

num_steps = 50
num_inputs = X_train.shape[2]

# create new inctance of the SNN Class
model = Net(num_inputs, num_steps).to(device)

model.load_state_dict(torch.load("trained_models/snn_60min_quant50", map_location=torch.device('cpu')))
model.eval()
actuals = []
predictions = []

with torch.no_grad():
    for data, targets in valid_loader:

        # prepare data
        data = data.to(device)
        targets = targets.to(device)

        data = data.float()
        targets = targets.float()

        _, _, _, output = model(data)

        output = output.cpu()
        output = output.squeeze(1).detach()

        prediction = output[-1]

        actuals.append(targets)
        predictions.append(prediction)

# Convert lists to tensors
actuals = torch.cat(actuals, dim=0)
predictions = torch.cat(predictions, dim=0)

mv1["power_pred_med"] = predictions[:, 0]
mv1["voltage_pred_med"] = predictions[:, 1]
mv1["current_pred_med"] = predictions[:, 2]
#mv1 = mv1.loc[(mv1.index > '2022-01-04') & (mv1.index < '2022-01-06')]
mv1 = mv1.loc[(mv1.index > '2021-12-12') & (mv1.index < '2021-12-14')]
mv2 = mv1

/var/folders/1g/jt7x_x9n6j1f3317_dwv3k9m0000gn/T/ipykernel_36727/3182949539.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("trained_mo

###  3.2 SNN Peformance Metrics

In [150]:
from keras.models import load_model

# Set parameters
batchsize_list = [300, 150, 50, 20, 8]
time_frame_list = ['3min', '5min', '15min', '30min', '60min']
time_frame_seconds_list = [180, 300, 900, 1800, 3600]
n = 0

snn_power_mape_list = []
snn_volt_mape_list = []
snn_curr_mape_list = []

# Dictionary to store mv variables
mv_dict = {}

for j in range(len(batchsize_list)):
    batchsize = batchsize_list[j]
    time_frame = time_frame_list[j]
    time_frame_seconds = time_frame_seconds_list[j]

    X = pd.concat([df["power - 1h"], df["power - 2h"], df["power - 3h"], 
                   df["V1 - 1h"], df["V1 - 2h"], df["V1 - 3h"], 
                   df["I1L - 1h"], df["I1L - 2h"], df["I1L - 3h"], 
                   df["EC - 1h"], df["EC - 2h"], df["EC - 3h"], 
                   df["raw_VWC - 1h"], df["raw_VWC - 2h"], df["raw_VWC - 3h"], 
                   df["temp - 1h"], df["temp - 2h"], df["temp - 3h"], 
                   df["tsd"], df["hour"]], axis=1)
    y = pd.concat([df["Power (uW)"], df['Voltage (mV)'], df['Current (uA)']], axis=1)

    # Normalize Data
    X_normalized = ((X - X.min()) / (X.max() - X.min()))

    # Split train and test sets
    X_train, X_test = train_test_split(X_normalized, test_size=0.3, shuffle=False)
    y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False)

    X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
    y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)

    # Calculate actual energy generated in test set
    E_actual = 0
    for i in range(len(y_test) - 1):
        t = (y_test.index[i+1] - y_test.index[i]).total_seconds()
        if t < 180:
            E_actual += y_test['Power (uW)'][i] * t

    # Resample data
    X_valid = X_valid.resample(time_frame).mean().dropna()
    y_valid = y_valid.resample(time_frame).mean().dropna()

    X_test = X_test.resample(time_frame).mean().dropna()
    y_test = y_test.resample(time_frame).mean().dropna()

    # Define mv variable for the current time frame
    mv_dict[time_frame] = y_test

    # Reshape data
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_valid = X_valid.values.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Convert to tensor
    X_train = torch.tensor(X_train)
    y_train = torch.tensor(y_train.values)
    X_valid = torch.tensor(X_valid)
    y_valid = torch.tensor(y_valid.values)
    X_test = torch.tensor(X_test)
    y_test = torch.tensor(y_test.values)

    # Make datasets
    train_dataset = TensorDataset(X_train, y_train)
    valid_dataset = TensorDataset(X_valid, y_valid)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=False)
    valid_loader = DataLoader(valid_dataset, batch_size=batchsize, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

    num_steps = 50
    num_inputs = X_train.shape[2]

    # Create new instance of the SNN Class
    model = Net(num_inputs, num_steps).to(device)

    file = 'trained_models/snn_' + time_frame + '_quant50.pth'
    print(file)

    checkpoint = torch.load(file, map_location=torch.device('cpu'), weights_only=True)
    model.load_state_dict(checkpoint['state_dict'])

    model.eval()
    actuals = []
    predictions = []

    with torch.no_grad():
        for data, targets in test_loader:
            # Prepare data
            data = data.to(device).float()
            targets = targets.to(device).float()

            _, _, _, output = model(data)

            output = output.cpu().squeeze(1).detach()
            actuals.append(targets)
            predictions.append(output[-1])

    # Convert lists to tensors
    actuals = torch.cat(actuals, dim=0)
    predictions = torch.cat(predictions, dim=0)

    mv = mv_dict[time_frame]
    mv["power_pred_med_" + time_frame] = predictions[:, 0].numpy()
    mv["voltage_pred_med_" + time_frame] = predictions[:, 1].numpy()
    mv["current_pred_med_" + time_frame] = predictions[:, 2].numpy()

    print(f'Voltage overestimation rate for {time_frame}: %.3f%%' % (
        (mv['Voltage (mV)'].values <= mv["voltage_pred_med_" + time_frame]).mean() * 100))
    print(f"Test MAPE power ({time_frame}): %3f" % MAPE(mv['Power (uW)'].values.ravel(), mv["power_pred_med_" + time_frame]))
    print(f"Test MAPE voltage ({time_frame}): %3f" % MAPE(mv['Voltage (mV)'], mv["voltage_pred_med_" + time_frame]))
    print(f"Test MAPE current ({time_frame}): %3f" % MAPE(mv['Current (uA)'], mv["current_pred_med_" + time_frame]))

    E_pred = 0
    for i in range(len(mv) - 1):
        t = (mv.index[i+1] - mv.index[i]).total_seconds()
        if t <= time_frame_seconds + 50:
            E_pred += mv["power_pred_med_" + time_frame][i] * t

    print(f'Predicted vs. Actual Total Energy Percent Difference ({time_frame}): %.3f%%' % (
        (E_pred - E_actual) * 100 / E_actual))

    V_actual = mv['Voltage (mV)'].mean()
    V_pred = mv["voltage_pred_med_" + time_frame].mean()
    print(f'Predicted vs. Actual Total Voltage Percent Difference ({time_frame}): %.3f%%' % (
        (V_pred - V_actual) * 100 / V_actual))


trained_models/snn_3min_quant50.pth
Voltage overestimation rate for 3min: 85.231%
Test MAPE power (3min): 0.537724
Test MAPE voltage (3min): 0.290864
Test MAPE current (3min): 0.316784
Predicted vs. Actual Total Energy Percent Difference (3min): 23.419%
Predicted vs. Actual Total Voltage Percent Difference (3min): 21.857%
trained_models/snn_5min_quant50.pth
Voltage overestimation rate for 5min: 73.179%
Test MAPE power (5min): 0.394733
Test MAPE voltage (5min): 0.214422
Test MAPE current (5min): 0.241269
Predicted vs. Actual Total Energy Percent Difference (5min): 16.691%
Predicted vs. Actual Total Voltage Percent Difference (5min): 12.812%
trained_models/snn_15min_quant50.pth
Voltage overestimation rate for 15min: 80.689%
Test MAPE power (15min): 0.328628
Test MAPE voltage (15min): 0.257437
Test MAPE current (15min): 0.227218
Predicted vs. Actual Total Energy Percent Difference (15min): 17.349%
Predicted vs. Actual Total Voltage Percent Difference (15min): 17.903%
trained_models/snn_30

In [214]:
print(mv_dict)

{'3min':                            Power (uW)  Voltage (mV)  Current (uA)   
timestamp                                                           
2022-01-04 00:30:00-08:00  560.181385   1833.837429   2936.854571  \
2022-01-04 00:33:00-08:00  344.316142   1675.286143   2122.057714   
2022-01-04 00:36:00-08:00  259.808313   1733.951462   1479.855846   
2022-01-04 00:39:00-08:00  340.043036   1747.128077   1866.355385   
2022-01-04 00:42:00-08:00  397.022258   1743.545692   2208.527462   
...                               ...           ...           ...   
2022-01-26 23:45:00-08:00  159.688701    892.816286   1724.953429   
2022-01-26 23:48:00-08:00  153.143727    805.634462   2100.996769   
2022-01-26 23:51:00-08:00  193.779752    872.273692   2170.535000   
2022-01-26 23:54:00-08:00  179.846046    854.210154   2092.793308   
2022-01-26 23:57:00-08:00  932.169339    992.999231   5453.417308   

                           power_pred_med_3min  voltage_pred_med_3min   
timestamp           

# Student Model Architecture

In [ ]:
# beta = 0.9

# # old design network
# # model = Sequential()
# # model.add(LSTM(200, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
# # model.add(Dense(100, activation='relu'))
# # model.add(Dense(3))
# # model.compile(loss=quantile_loss, metrics=['mape'], optimizer='adam')

# # Define Network
# class StudentModel(nn.Module):
#     """
#         Args:
#             input_size (int): Number of input features (e.g., raw features + teacher predictions).
#             hidden_size (int): Number of hidden units in each LSTM layer.
#             output_size (int): Number of output features (e.g., predicting Power, Voltage, Current for 15, 30, 60 min).
#             num_layers (int): Number of LSTM layers.
#             dropout (float): Dropout rate for regularization.
#     """

#     def __init__(self, num_inputs, num_steps, output_size):
#         super(StudentModel, self).__init__()

#         self.num_inputs = num_inputs
#         self.num_steps = num_steps
#         self.output_size = output_size

#         # LSTM layer (Temporal modeling)
#         self.lstm = nn.LSTM(input_size=num_inputs, hidden_size=200, num_layers=1, batch_first=True)

#         # Fully connected layers (Feed-forward modeling)
#         self.fc1 = nn.Linear(200, 100)  # Map LSTM output to intermediate features.
#         self.fc2 = nn.Linear(100, output_size)  # Map intermediate features to final outputs.

#         # Activation functions
#         self.relu = nn.ReLU()
        
#     def forward(self, x):
#         """
#         Forward pass through the student model.
#         Args:
#             x (torch.Tensor): Input tensor of shape (batch_size, num_steps, num_inputs).
#         Returns:
#             torch.Tensor: Predictions for the specified time horizons.
#         """
#         # LSTM forward pass
#         lstm_out, _ = self.lstm(x)  # Output shape: (batch_size, num_steps, hidden_size)
#         lstm_out = lstm_out[:, -1, :]  # Take the last time step's output.

#         # Fully connected layers
#         fc1_out = self.relu(self.fc1(lstm_out))
#         output = self.fc2(fc1_out)

#         return output

## Prepare Data for Student Model

In [198]:
# Load teacher predictions
timeframe = '5min'
teacher_5min_preds = mv_dict[timeframe]
teacher_5min_preds_df = pd.DataFrame(teacher_5min_preds, columns=["teacher_power_" + timeframe, "teacher_voltage_" + timeframe, "teacher_current_" + timeframe])

# Reset indices for both DataFrames
X = X.reset_index(drop=True)
teacher_5min_preds_df = teacher_5min_preds_df.reset_index(drop=True)

# Concatenate along the columns
X_with_teacher = pd.concat([X, teacher_5min_preds_df], axis=1)

# Verify the updated shape
print("Updated X Shape: Rows " + str(X_with_teacher.shape[0]) + " Columns " + str(X_with_teacher.shape[1]))

Updated X Shape: Rows 969652 Columns 23


In [217]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_with_teacher, y, test_size=0.2, random_state=42)

# Verify the shapes of the splits
print("Training set shape (X_train):", X_train.shape)
print("Training labels shape (y_train):", y_train.shape)
print("Test set shape (X_test):", X_test.shape)
print("Test labels shape (y_test):", y_test.shape)

Training set shape (X_train): (775721, 23)
Training labels shape (y_train): (775721, 3)
Test set shape (X_test): (193931, 23)
Test labels shape (y_test): (193931, 3)


## Train Student model

In [222]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming you have your dataset X and y loaded

# Define quantile loss function
def quantile_loss(y_true, y_pred, quantile=0.5):
    error = y_true - y_pred
    loss = torch.mean(torch.max(quantile * error, (quantile - 1) * error))
    return loss

# Define the StudentModel (as provided earlier)
class StudentModel(nn.Module):
    def __init__(self, num_inputs, num_steps, output_size):
        super(StudentModel, self).__init__()
        self.num_inputs = num_inputs
        self.num_steps = num_steps
        self.output_size = output_size

        self.lstm = nn.LSTM(input_size=num_inputs, hidden_size=200, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(200, 100)
        self.fc2 = nn.Linear(100, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]
        fc1_out = self.relu(self.fc1(lstm_out))
        output = self.fc2(fc1_out)
        return output

# Set parameters
batch_size = 32  # Adjust as needed
num_epochs = 25
learning_rate = 1e-3
beta = 0.3  # For quantile loss

# Normalize Data
X_normalized = ((X - X.min()) / (X.max() - X.min()))

# Split train and test sets
X_train, X_test = train_test_split(X_normalized, test_size=0.3, shuffle=False)
y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False)

# Split the test set further into validation and test sets
X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)

# Reshape data for LSTM input
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_valid = X_valid.values.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert to tensor
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train.values).float()
X_valid = torch.tensor(X_valid).float()
y_valid = torch.tensor(y_valid.values).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test.values).float()

# Create datasets
train_dataset = TensorDataset(X_train, y_train)
valid_dataset = TensorDataset(X_valid, y_valid)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define model, optimizer, and loss function
num_steps = 1  # Since you're using LSTM for time series data with one step
num_inputs = X_train.shape[2]
output_size = y_train.shape[1]
model = StudentModel(num_inputs, num_steps, output_size).to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(data)

        # Compute the loss
        loss = quantile_loss(targets, outputs, quantile=0.5)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()

        if i % 10 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Batch {i}/{len(train_loader)}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {running_loss / len(train_loader):.4f}")

# Validation loop
model.eval()
valid_loss = 0.0
with torch.no_grad():
    for data, targets in valid_loader:
        data = data.to(device)
        targets = targets.to(device)
        
        outputs = model(data)
        loss = quantile_loss(targets, outputs, quantile=0.5)
        valid_loss += loss.item()

print(f"Validation Loss: {valid_loss / len(valid_loader):.4f}")

# Test loop
test_loss = 0.0
with torch.no_grad():
    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(device)
        
        outputs = model(data)
        loss = quantile_loss(targets, outputs, quantile=0.5)
        test_loss += loss.item()

print(f"Test Loss: {test_loss / len(test_loader):.4f}")


Epoch 1/25, Batch 0/21212, Loss: 11510.8154
Epoch 1/25, Batch 10/21212, Loss: 11551.8818
Epoch 1/25, Batch 20/21212, Loss: 11358.7432
Epoch 1/25, Batch 30/21212, Loss: 11602.2549
Epoch 1/25, Batch 40/21212, Loss: 11697.6250
Epoch 1/25, Batch 50/21212, Loss: 11584.3516
Epoch 1/25, Batch 60/21212, Loss: 11595.3984
Epoch 1/25, Batch 70/21212, Loss: 11659.2920
Epoch 1/25, Batch 80/21212, Loss: 11482.1338
Epoch 1/25, Batch 90/21212, Loss: 11528.4834
Epoch 1/25, Batch 100/21212, Loss: 11485.4756
Epoch 1/25, Batch 110/21212, Loss: 797.7330
Epoch 1/25, Batch 120/21212, Loss: 7652.8218
Epoch 1/25, Batch 130/21212, Loss: 8966.7217
Epoch 1/25, Batch 140/21212, Loss: 9524.6973
Epoch 1/25, Batch 150/21212, Loss: 8974.8330
Epoch 1/25, Batch 160/21212, Loss: 8777.5889
Epoch 1/25, Batch 170/21212, Loss: 8549.2861
Epoch 1/25, Batch 180/21212, Loss: 7184.4199
Epoch 1/25, Batch 190/21212, Loss: 6319.5571
Epoch 1/25, Batch 200/21212, Loss: 6364.0044
Epoch 1/25, Batch 210/21212, Loss: 6646.7603
Epoch 1/25,

In [ ]:
# Validation loop
model.eval()
valid_loss = 0.0
valid_mape = 0.0
valid_mse = 0.0

with torch.no_grad():
    for data, targets in valid_loader:
        data = data.to(device)
        targets = targets.to(device)
        
        outputs = model(data)
        loss = quantile_loss(targets, outputs, quantile=0.5)
        valid_loss += loss.item()
        
        # Calculate MAPE
        mape = mean_absolute_percentage_error(targets.cpu().numpy(), outputs.cpu().numpy())
        valid_mape += mape
        
        # Calculate MSE
        mse = mean_squared_error(targets.cpu().numpy(), outputs.cpu().numpy())
        valid_mse += mse

print(f"Validation Loss: {valid_loss / len(valid_loader):.4f}")
print(f"Validation MAPE: {valid_mape / len(valid_loader):.4f}")
print(f"Validation MSE: {valid_mse / len(valid_loader):.4f}")

# Test loop
test_loss = 0.0
test_mape = 0.0
test_mse = 0.0

with torch.no_grad():
    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(device)
        
        outputs = model(data)
        loss = quantile_loss(targets, outputs, quantile=0.5)
        test_loss += loss.item()
        
        # Calculate MAPE
        mape = mean_absolute_percentage_error(targets.cpu().numpy(), outputs.cpu().numpy())
        test_mape += mape
        
        # Calculate MSE
        mse = mean_squared_error(targets.cpu().numpy(), outputs.cpu().numpy())
        test_mse += mse

print(f"Test Loss: {test_loss / len(test_loader):.4f}")
print(f"Test MAPE: {test_mape / len(test_loader):.4f}")
print(f"Test MSE: {test_mse / len(test_loader):.4f}")

Validation Loss: 532.0240
Validation MAPE: 52762973149808.4688
Validation MSE: 3174036.3891
Test Loss: 529.5674
Test MAPE: 7.1660
Test MSE: 3138549.0116


In [ ]:

batchsize_list = [300, 150, 50, 20, 8]
time_frame_list = ['3min', '5min', '15min', '30min', '60min']
time_frame_seconds_list = [180, 300, 900, 1800, 3600]
n = 0

snn_power_mape_list = []
snn_volt_mape_list = []
snn_curr_mape_list = []

# Dictionary to store mv variables
mv_dict = {}

for j in range(len(batchsize_list)):
    batchsize = batchsize_list[j]
    time_frame = time_frame_list[j]
    time_frame_seconds = time_frame_seconds_list[j]

    X = pd.concat([df["power - 1h"], df["power - 2h"], df["power - 3h"], 
                   df["V1 - 1h"], df["V1 - 2h"], df["V1 - 3h"], 
                   df["I1L - 1h"], df["I1L - 2h"], df["I1L - 3h"], 
                   df["EC - 1h"], df["EC - 2h"], df["EC - 3h"], 
                   df["raw_VWC - 1h"], df["raw_VWC - 2h"], df["raw_VWC - 3h"], 
                   df["temp - 1h"], df["temp - 2h"], df["temp - 3h"], 
                   df["tsd"], df["hour"]], axis=1)
    y = pd.concat([df["Power (uW)"], df['Voltage (mV)'], df['Current (uA)']], axis=1)

    # Normalize Data
    X_normalized = ((X - X.min()) / (X.max() - X.min()))

    # Split train and test sets
    X_train, X_test = train_test_split(X_normalized, test_size=0.3, shuffle=False)
    y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False)

    X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
    y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)

    # Calculate actual energy generated in test set
    E_actual = 0
    for i in range(len(y_test) - 1):
        t = (y_test.index[i+1] - y_test.index[i]).total_seconds()
        if t < 180:
            E_actual += y_test['Power (uW)'][i] * t

    # Resample data
    X_valid = X_valid.resample(time_frame).mean().dropna()
    y_valid = y_valid.resample(time_frame).mean().dropna()

    X_test = X_test.resample(time_frame).mean().dropna()
    y_test = y_test.resample(time_frame).mean().dropna()

    # Define mv variable for the current time frame
    mv_dict[time_frame] = y_test

    # Reshape data
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_valid = X_valid.values.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Convert to tensor
    X_train = torch.tensor(X_train)
    y_train = torch.tensor(y_train.values)
    X_valid = torch.tensor(X_valid)
    y_valid = torch.tensor(y_valid.values)
    X_test = torch.tensor(X_test)
    y_test = torch.tensor(y_test.values)

    # Make datasets
    train_dataset = TensorDataset(X_train, y_train)
    valid_dataset = TensorDataset(X_valid, y_valid)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=False)
    valid_loader = DataLoader(valid_dataset, batch_size=batchsize, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

    num_steps = 50
    num_inputs = X_train.shape[2]

    # Create new instance of the SNN Class
    model = Net(num_inputs, num_steps).to(device)

    file = 'trained_models/snn_' + time_frame + '_quant50.pth'
    print(file)

    checkpoint = torch.load(file, map_location=torch.device('cpu'), weights_only=True)
    model.load_state_dict(checkpoint['state_dict'])

    model.eval()
    actuals = []
    predictions = []

    with torch.no_grad():
        for data, targets in test_loader:
            # Prepare data
            data = data.to(device).float()
            targets = targets.to(device).float()

            _, _, _, output = model(data)

            output = output.cpu().squeeze(1).detach()
            actuals.append(targets)
            predictions.append(output[-1])

    # Convert lists to tensors
    actuals = torch.cat(actuals, dim=0)
    predictions = torch.cat(predictions, dim=0)

    mv = mv_dict[time_frame]
    mv["power_pred_med_" + time_frame] = predictions[:, 0].numpy()
    mv["voltage_pred_med_" + time_frame] = predictions[:, 1].numpy()
    mv["current_pred_med_" + time_frame] = predictions[:, 2].numpy()

    print(f'Voltage overestimation rate for {time_frame}: %.3f%%' % (
        (mv['Voltage (mV)'].values <= mv["voltage_pred_med_" + time_frame]).mean() * 100))
    print(f"Test MAPE power ({time_frame}): %3f" % MAPE(mv['Power (uW)'].values.ravel(), mv["power_pred_med_" + time_frame]))
    print(f"Test MAPE voltage ({time_frame}): %3f" % MAPE(mv['Voltage (mV)'], mv["voltage_pred_med_" + time_frame]))
    print(f"Test MAPE current ({time_frame}): %3f" % MAPE(mv['Current (uA)'], mv["current_pred_med_" + time_frame]))

    E_pred = 0
    for i in range(len(mv) - 1):
        t = (mv.index[i+1] - mv.index[i]).total_seconds()
        if t <= time_frame_seconds + 50:
            E_pred += mv["power_pred_med_" + time_frame][i] * t

    print(f'Predicted vs. Actual Total Energy Percent Difference ({time_frame}): %.3f%%' % (
        (E_pred - E_actual) * 100 / E_actual))

    V_actual = mv['Voltage (mV)'].mean()
    V_pred = mv["voltage_pred_med_" + time_frame].mean()
    print(f'Predicted vs. Actual Total Voltage Percent Difference ({time_frame}): %.3f%%' % (
        (V_pred - V_actual) * 100 / V_actual))
